In [1]:
import os
import dotenv
from typing import Tuple

from base64 import b64decode
from algosdk.future import transaction
from algosdk.v2client.algod import AlgodClient
from algosdk.logic import get_application_address
from pyteal import compileTeal, Mode, Expr

from contracts.validator import AllyValidator
from utils import wait_for_transaction, get_algod_client, get_genesis_accounts, get_balances
from account import Account

dotenv.load_dotenv(".env")

True

In [2]:
client = get_algod_client(os.environ.get("ALGOD_URL"), os.environ.get("ALGOD_API_KEY"))

genesis_accounts = get_genesis_accounts(os.environ.get("KMD_URL"), os.environ.get("KMD_API_KEY"))
for idx, account in enumerate(genesis_accounts):
    print(f"Account{idx + 1}: {account.get_address()}")

creator = Account.from_mnemonic(os.environ.get("CREATOR_MNEMONIC"))
print(f"Creator: {creator.get_address()}")

Account1: ISACMK5G7S42XMCIQ76TSEHWYJYM3PBMABLCVDVRJHTGFEH3UFLQYECJKI
Account2: U4DBBBYDSVLEGHQIDYULSLDMFO2NT22LE7XRQWRNQF4F5RPR5TZ5X4TCOU
Account3: XVVY5KWWKEPBZHKLPRFY52ZUH4WZ5MLQ4GPODPPBWQDWIDZ5LWVIGUO6PE
Creator: U22M6ENJLHAQEIL6TVCC3BJFM63JLFVH4ZFS3XYPA3XRUCW3NN5MKC5YVU


In [3]:
def send_algo(client: AlgodClient, sender: Account, receiver_addr: str, amount: int):
    txn = transaction.PaymentTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        receiver=receiver_addr,
        amt=amount
    )
    signd_txn = txn.sign(sender.get_private_key())
    client.send_transaction(signd_txn)
    
    wait_for_transaction(client, signd_txn.get_txid())

In [4]:
creator_balance = get_balances(client, creator.get_address())
if creator_balance.get(0) == 0:
    # send 10 algo to creator
    send_algo(client, genesis_accounts[0], creator.get_address(), 10_000_000)

Waiting for confirmation...
Transaction 2JI7TM3KLJXLLBTJOOOMPP5A5KRY334LPWFQQAJEQ7BHB4QHWJHA confirmed in round 6.


In [5]:
def fullyCompileContract(client: AlgodClient, contract: Expr) -> bytes:
    teal = compileTeal(contract, mode=Mode.Application, version=5)
    response = client.compile(teal)
    return b64decode(response["result"])


def get_contracts(client: AlgodClient) -> Tuple[bytes, bytes]:
    simple_contract = AllyValidator()

    approval_program = fullyCompileContract(
        client, simple_contract.approval_program())
    clear_state_program = fullyCompileContract(
        client, simple_contract.clear_state_program())

    return approval_program, clear_state_program


def create_pool(client: AlgodClient, creator: Account):
    approval, clear = get_contracts(client)
    global_schema = transaction.StateSchema(num_uints=7, num_byte_slices=2)
    local_schema = transaction.StateSchema(num_uints=0, num_byte_slices=0)

    txn = transaction.ApplicationCreateTxn(
        sender=creator.get_address(),
        sp=client.suggested_params(),
        on_complete=transaction.OnComplete.NoOpOC,
        approval_program=approval,
        clear_program=clear,
        global_schema=global_schema,
        local_schema=local_schema,
    )
    signed_txn = txn.sign(creator.get_private_key())
    client.send_transaction(signed_txn)

    response = wait_for_transaction(client, signed_txn.get_txid())
    assert response.application_index is not None and response.application_index > 0
    return response.application_index


app_id = create_pool(client, creator)

print(f"App ID: {app_id}")
print(f"App address: {get_application_address(app_id)}")

Waiting for confirmation...
Transaction 2V56IV7QH36EXU2I7XHCEIPTO3EP6YFLNH5ZEJLZ5DHJ6PKQ65UA confirmed in round 8.
App ID: 2
App address: FHQPLJVRO7FVHYKCA2SGR3I7ZHHKHAQS4AGOAVGGGJQJYTTBVEJUGHN5JQ


In [6]:
app_balance = get_balances(client, get_application_address(app_id))
if app_balance.get(0) == 0:
    # send 0.201 algo to app
    send_algo(client, creator, get_application_address(app_id), 201_000)

def bootstrap_pool(client: AlgodClient, sender: Account, application_id: int):
    txn = transaction.ApplicationCallTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=application_id,
        on_complete=transaction.OnComplete.NoOpOC,
        app_args=[
            b"bootstrap"
        ]
    )
    signed_txn = txn.sign(sender.get_private_key())
    client.send_transaction(signed_txn)

    wait_for_transaction(client, signed_txn.get_txid())
    

bootstrap_pool(client, creator, app_id)

Waiting for confirmation...
Transaction UEJU4GOCP3BSZNFQA5BMU5MLZBLH2ADKTJKNCNAKMDFIP5T23YCQ confirmed in round 10.
Waiting for confirmation...
Transaction NU5JG2SBLJSTKIJNQYJIX72BT4CI63FX3OU26YTQGV6IPTWP225A confirmed in round 12.


In [7]:
from utils import get_app_global_state

state = get_app_global_state(client, app_id)
print("global state:", state)
print("app balance:", get_balances(client, get_application_address(app_id)))

walgo_id = state.get(b"token_id")

global state: {b'owner': b'\xa6\xb4\xcf\x11\xa9Y\xc1\x02!~\x9dD-\x85%g\xb6\x95\x96\xa7\xe6K-\xdf\x0f\x06\xef\x1a\n\xdbkz', b'token_id': 5, b'total_mint_amt': 0}
app balance: {0: 200000, 5: 18446744073709551615}


In [8]:
def asset_is_opted_in(client: AlgodClient, addr: str, asset_id: int):
    account_info = client.account_info(addr)
    for a in account_info.get('assets', []):
        if a['asset-id'] == asset_id:
            return True
    return False


def asset_opt_in(client: AlgodClient, sender: Account, asset_id: int):
    txn = transaction.AssetOptInTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=asset_id
    )
    signed_txn = txn.sign(sender.get_private_key())
    tx_id = client.send_transaction(signed_txn)
    
    wait_for_transaction(client, tx_id)


def mint_walgo(client: AlgodClient, sender: Account, application_id: int, amount: int, asset_id: int):
    payment_txn = transaction.PaymentTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        receiver=get_application_address(application_id),
        amt=amount
    )
    call_txn = transaction.ApplicationCallTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=application_id,
        on_complete=transaction.OnComplete.NoOpOC,
        app_args=[
            b"mint"
        ],
        foreign_assets=[asset_id]
    )
    transaction.assign_group_id([payment_txn, call_txn])
    
    signed_payment_txn = payment_txn.sign(sender.get_private_key())
    signed_call_txn = call_txn.sign(sender.get_private_key())
    tx_id = client.send_transactions([signed_payment_txn, signed_call_txn])
    
    wait_for_transaction(client, tx_id)


minter = genesis_accounts[0]
if not asset_is_opted_in(client, minter.get_address(), walgo_id):
    asset_opt_in(client, minter, walgo_id)

mint_walgo(client, minter, app_id, 10_000_000, walgo_id)
print("global state:", get_app_global_state(client, app_id))
print("app balance:", get_balances(client, get_application_address(app_id)))
print("minter balance:", get_balances(client, minter.get_address()))

Waiting for confirmation...
Transaction OLCUF7DHZAORZ6RR43OQKZXGXVXY5MZSFRWC7VZE3DHL6FH7IGYQ confirmed in round 14.
Waiting for confirmation...
Transaction FSAUZ35NVC2LFKGUQD6X65MPLQY33NFFZBAYIJXCXR4VC3ROWBQQ confirmed in round 16.
global state: {b'owner': b'\xa6\xb4\xcf\x11\xa9Y\xc1\x02!~\x9dD-\x85%g\xb6\x95\x96\xa7\xe6K-\xdf\x0f\x06\xef\x1a\n\xdbkz', b'total_mint_amt': 10000000, b'token_id': 5}
app balance: {0: 10199000, 5: 18446744073699551615}
minter balance: {0: 3999999979996000, 5: 10000000}


In [9]:
mint_walgo(client, minter, app_id, 1_500_000, walgo_id)

print("global state:", get_app_global_state(client, app_id))
print("app balance:", get_balances(client, get_application_address(app_id)))
print("minter balance:", get_balances(client, minter.get_address()))

Waiting for confirmation...
Transaction 34KIK6TDJPUXV5KJFEMJGT3FOJ5DJFFKTWISKRJRQCMRQHDQOFPA confirmed in round 18.
global state: {b'owner': b'\xa6\xb4\xcf\x11\xa9Y\xc1\x02!~\x9dD-\x85%g\xb6\x95\x96\xa7\xe6K-\xdf\x0f\x06\xef\x1a\n\xdbkz', b'token_id': 5, b'total_mint_amt': 11754850}
app balance: {0: 11698000, 5: 18446744073697796765}
minter balance: {0: 3999999978494000, 5: 11754850}
